## Compare isotope production with KamLAND FLUKA simulation results

There are two different files: created by residnuc where the first line consists of the isotopes counting lists and the mgdraw srcefiles where the first line includes the neutron count (however, this is not the case for the new usdraw version anylonger)

In [2]:
import ast
from itertools import count 
import sys

In [3]:
def AZlist(list_name):
    isotope_list = []
    for item in list_name:
        Z, A, count = item
        isotope_list.append([Z, A])
    return isotope_list


def ktonday(no_muons, eventcount):
    # express count as (kton day)^-1
    # no_muons_simulated should change according to the simulation

    sec24hr = 86400. 
    muonrate = 0.198 # Hz
    frequency = 1. / muonrate # every 5 sec one muon passes through
    time_muons = (no_muons * frequency) / sec24hr # 585 days of simulating, detector livetime

    flukatracklength = 2500. #cm
    meantracklength = 874. #cm
    fluka_livetime = time_muons * (flukatracklength / meantracklength)

    densityXeLS = 0.78013 #g/cm^3
    XeLSvolume = 1150
    XeLSmass = densityXeLS * XeLSvolume * 10**(-3) # to get kton
    
    ktonday = eventcount / (XeLSmass * fluka_livetime) # (kton day)^-1

    return ktonday

In [18]:
# version 2 Kelly Weerman
# compare the isotope production txt file with KamLAND 

def read_kamlandfile(file_name):
    # read the zen800 txt file and return Z, A, count list
    
    kamland_file = open(file_name, 'r')
    kamland_list, error_list = [], []
    for line in kamland_file:
        Z, A, count, error = line.split()
        kamland_list.append([int(Z), int(A), float(count)])
        error_list.append(float(error))
    kamland_file.close()

    return kamland_list, error_list


def compare_KamLAND(file_names, newfile_name, no_muons):
    # compare isotope results with kamland file
    # file_names = ["ownresults.txt, kamlandresults.txt"]

    newfile = open(newfile_name, "w")
    file_kelly = open(file_names[0], 'r')

    # check if the header of the file contains neutron count
    line1 = file_kelly.readline()
    if len(line1) < 100:
        neutrons = line1.split()[5]
        print(neutrons)

        # skip to the next line containing the lists
        isotopes_kelly = file_kelly.readline().strip()
    else:
        isotopes_kelly = line1.strip()

    list_kelly = ast.literal_eval(isotopes_kelly)
    isotopes_kelly = AZlist(list_kelly)

    # the count list of kamland is created here, note count is in events/day/ktonLS
    list_kamland, list_error = read_kamlandfile(file_names[1])
    isotopes_kamland = AZlist(list_kamland)
    
    newfile.write("Comparison {0} - {1} \n".format(file_names[0], file_names[1]))
    newfile.write("Index: Z, A, count {0}, count {1}, difference, error KamLAND, difference / count of {0} \n".format(file_names[0], file_names[1]))

    for isotope in isotopes_kelly:
        index_kelly = isotopes_kelly.index(isotope)
        count_kelly = ktonday(no_muons, list_kelly[index_kelly][2])

        if isotope in isotopes_kamland:
            index_kamland = isotopes_kamland.index(isotope)
            count_kamland = list_kamland[index_kamland][2]
            difference = count_kelly - count_kamland
            percentage = (float(difference)/count_kamland) * 100
            error_kamland = list_error[index_kamland]
            newfile.write("{0} {1} {2:e} {3:e} {4:e}, error: {5:e}, in percentage {6:.2f} \n"\
                .format(isotope[0], isotope[1], count_kelly, count_kamland, difference, error_kamland, (percentage)))
        else:
            newfile.write("{0} {1} {2:e} not in file {3}\n"\
                .format(isotope[0], isotope[1], count_kelly, file_names[1]))
        
    for isotope in isotopes_kamland:
        index_kamland = isotopes_kamland.index(isotope)
        count_kamland = list_kamland[index_kamland][2]

        if isotope not in isotopes_kelly:
            newfile.write("{0} {1} {2:e} not in file {3}\n"\
                .format(isotope[0], isotope[1], count_kamland, file_names[0]))
            
    file_kelly.close()
    newfile.close()

no_muons = 10000000
#zen800_210602
#zen800_newversion
isotopes_path = "/project/xenon/kweerman/exercises/XeLS_results_isotopes/txt_isotopes/"
compare_path = "/project/xenon/kweerman/exercises/XeLS_results_isotopes/txt_compare/"
file_names = [isotopes_path + "AZisotopesUnform.txt", "/project/xenon/kweerman/exercises/zen800_newversion.txt"]

compare_KamLAND(file_names, compare_path + "ComparisonUnformFLUKA2021.txt", no_muons)

## Compare two isotope files

In [7]:
def read_kelly_file(file_name):
    # read the first (two) lines of the txt file
    line1 = file_name.readline()
    if len(line1) < 100:
        neutrons = line1.split()[5]
        print(neutrons)
        
        # skip to the next line containing the lists
        isotopes = file_name.readline().strip()
    else:
        isotopes = line1.strip()
        
    return isotopes

def compare_kelly_files(file_names, newfile_name, no_muons):
    # compare isotope results with other isotope files
    # file_names = ["ownresults.txt, other_results.txt"]

    newfile = open(newfile_name, "w")
    file_kelly1 = open(file_names[0], 'r')
    file_kelly2 = open(file_names[1], 'r')

    # check if the header of the file contains neutron count
    # and save the isotope lists
    isotopes_kelly1 = read_kelly_file(file_kelly1)
    isotopes_kelly2 = read_kelly_file(file_kelly2)

    list_kelly1 = ast.literal_eval(isotopes_kelly1)
    isotopes_kelly1 = AZlist(list_kelly1)
    list_kelly2 = ast.literal_eval(isotopes_kelly2)
    isotopes_kelly2 = AZlist(list_kelly2)
    
    newfile.write("Comparison {0} - {1} \n".format(file_names[0], file_names[1]))
    newfile.write("Index: Z, A, count {0}, count {1}, difference, difference / count of {0} \n".format(file_names[0], file_names[1]))

    for isotope in isotopes_kelly1:
        index_kelly1 = isotopes_kelly1.index(isotope)
        count_kelly1 = ktonday(no_muons, list_kelly1[index_kelly1][2])

        if isotope in isotopes_kelly2:
            index_kelly2 = isotopes_kelly2.index(isotope)
            count_kelly2 = ktonday(no_muons, list_kelly2[index_kelly2][2])
            difference = count_kelly1 - count_kelly2
            percentage = (float(difference)/count_kelly2) * 100
            newfile.write("{0} {1} {2:e} {3:e} {4:e}, in percentage {5:.2f} \n"\
                .format(isotope[0], isotope[1], count_kelly1, count_kelly2, difference, percentage))
        else:
            newfile.write("{0} {1} {2:e} not in file {3}\n"\
                .format(isotope[0], isotope[1], count_kelly1, file_names[1]))
        
    for isotope in isotopes_kelly2:
        index_kelly2 = isotopes_kelly2.index(isotope)
        count_kelly2 = ktonday(no_muons, list_kelly2[index_kelly2][2])

        if isotope not in isotopes_kelly1:
            newfile.write("{0} {1} {2:e} not in file {3}\n"\
                .format(isotope[0], isotope[1], count_kelly2, file_names[0]))
            
    file_kelly1.close()
    file_kelly2.close()
    newfile.close()

no_muons = 15000000
isotopes_path = "/project/xenon/kweerman/exercises/XeLS_results_isotopes/txt_isotopes/"
compare_path = "/project/xenon/kweerman/exercises/XeLS_results_isotopes/txt_compare/"
file_names = [isotopes_path + "AZisotopesNewFlukaAllPhysics.txt", isotopes_path + "AZisotopesNewFlukaAllPhysics2(diffseeds).txt"]

compare_kelly_files(file_names, compare_path + "ComparisonNF_diffseeds.txt", no_muons)

## Compare the two KamLAND versions with each other

In [17]:
# version 2 Kelly Weerman
# compare the isotope production txt file with KamLAND 

def read_kamlandfile(file_name):
    # read the zen800 txt file and return Z, A, count list
    
    kamland_file = open(file_name, 'r')
    kamland_list, error_list = [], []
    for line in kamland_file:
        Z, A, count, error = line.split()
        kamland_list.append([int(Z), int(A), float(count)])
        error_list.append(float(error))
    kamland_file.close()

    return kamland_list, error_list


def compare_KamLAND(file_names, newfile_name, no_muons):
    # compare isotope results with kamland file
    # file_names = ["ownresults.txt, kamlandresults.txt"]

    newfile = open(newfile_name, "w")
    file_kelly = open(file_names[0], 'r')

    list_kelly, kelly_error = read_kamlandfile(file_names[0])
    isotopes_kelly = AZlist(list_kelly)
    
    # the count list of kamland is created here, note count is in events/day/ktonLS
    list_kamland, list_error = read_kamlandfile(file_names[1])
    isotopes_kamland = AZlist(list_kamland)

    newfile.write("Comparison {0} - {1} \n".format(file_names[0], file_names[1]))
    newfile.write("Index: Z, A, count {0}, count {1}, difference, error KamLAND1, error KamLAND2, difference / count of {0} \n".format(file_names[0], file_names[1]))

    for isotope in isotopes_kelly:
        index_kelly = isotopes_kelly.index(isotope)
        count_kelly = list_kelly[index_kelly][2]

        if isotope in isotopes_kamland:
            index_kamland = isotopes_kamland.index(isotope)
            count_kamland = list_kamland[index_kamland][2]
            difference = count_kelly - count_kamland
            percentage = (float(difference)/count_kamland) * 100
            error_kamland = list_error[index_kamland]
            error_kelly = kelly_error[index_kelly]
            newfile.write("{0} {1} {2:e} {3:e} {4:e}, error1: {5:e}, error2: {6:e}, in percentage {7:.2f} \n"\
                .format(isotope[0], isotope[1], count_kelly, count_kamland, difference, error_kamland, error_kelly, (percentage)))
        else:
            newfile.write("{0} {1} {2:e} not in file {3}\n"\
                .format(isotope[0], isotope[1], count_kelly, file_names[1]))
        
    for isotope in isotopes_kamland:
        index_kamland = isotopes_kamland.index(isotope)
        count_kamland = list_kamland[index_kamland][2]

        if isotope not in isotopes_kelly:
            newfile.write("{0} {1} {2:e} not in file {3}\n"\
                .format(isotope[0], isotope[1], count_kamland, file_names[0]))
            
    file_kelly.close()
    newfile.close()

# find no_muons from the FlukaReader file, printed that
#no_muons = 15000000
#file_names = ["/project/xenon/kweerman/exercises/XResults_isotopes/txt_isotopes/AZisotopes21feb.txt", "/project/xenon/kweerman/exercises/zen800_newversion.txt"]
#compare_KamLAND(file_names, "/project/xenon/kweerman/exercises/Comparison21febFLUKA2021.txt", no_muons)
#no_muons = 10000000
#file_names = ["/project/xenon/kweerman/exercises/ZResults/Ieki/AZisotopesIeki.txt", "/project/xenon/kweerman/exercises/zen800_newversion.txt"]
#compare_KamLAND(file_names, "/project/xenon/kweerman/exercises/ComparisonIekiFLUKA2021.txt", no_muons)
no_muons = 15000000
file_names = ["/project/xenon/kweerman/exercises/zen800_210602.txt", "/project/xenon/kweerman/exercises/zen800_newversion.txt"]
compare_KamLAND(file_names, "/project/xenon/kweerman/exercises/test.txt", no_muons)

## Error calculation

In [ ]:
def ktonday(no_muons, eventcount):
    # express count as (kton day)^-1
    # no_muons_simulated should change according to the simulation

    sec24hr = 86400. 
    #muonrate = 0.198 # Hz
    muonrate = 0.198 +0.014
    frequency = 1. / muonrate # every 5 sec one muon passes through
    time_muons = (no_muons * frequency) / sec24hr # 585 days of simulating, detector livetime

    flukatracklength = 2500. #cm
    meantracklength = 874. - 13 #cm
    fluka_livetime = time_muons * (flukatracklength / meantracklength)

    densityXeLS = 0.78013 #g/cm^3
    XeLSvolume = 1150
    XeLSmass = densityXeLS * XeLSvolume * 10**(-3) # to get kton
    
    ktonday = eventcount / (XeLSmass * fluka_livetime) # (kton day)^-1
    factor = (XeLSmass * fluka_livetime)

    return ktonday

#ktonday(150*10**5, 8350749)
#3711 3500 3914
#3914-3500